## This is the simulation of the circuit on page 12

In [4]:
import sys
import os

sys.path.append(os.path.abspath("../project/qudit_cirq"))  

#### Description of the algorithm:



- Prepare the Generalized GHZ State (ψ₀): This is a multi-qudit entangled state, initialized for n clients.
- Teleportation Using a Generalized Bell State (|β⟩): The server teleports each qudit in ψ₀ to the clients.
- Client Actions:
Each client runs an LDP (Local Differential Privacy) Algorithm.
Applies a generalized Z gate based on the output of the LDP Algorithm.
Applies a generalized Hadamard (H) gate.
- Measurement: Each client measures their qudit in the computational basis.
Server Aggregation: The server collects the measurement results, computes a de-biased sum, and applies the formula provided.


In [5]:
import cirq
import numpy as np 

In [6]:
from qudit_cirq.qudit import quditXGate, quditZGate, quditHGate, quditCNOTGate, state_vector, measure_qudits

ModuleNotFoundError: No module named 'qudit_cirq'

In [24]:
def choose_d(n, kappa):
    return int(np.ceil((kappa - 1) * n)) + 1  

In [25]:
# Parameters
n_clients = 4  
kappa = 2  

d = choose_d(n_clients, kappa)
print(f"Chosen d: {d}")

gamma = kappa / (kappa - 1 + np.exp(1)) 
inputs = [1, 2, 3, 4] 

simulator = cirq.Simulator()

Chosen d: 5


In [26]:
def prepare_qudit_ghz_state(n_clients, d):
    """Prepares a generalized GHZ state with n_clients qudits of dimension d."""
    qudits = [cirq.LineQid(i, dimension=d) for i in range(n_clients)]
    circuit = cirq.Circuit()
    circuit.append(quditHGate(d).on(qudits[0]))  # Apply qudit Hadamard gate
    for i in range(1, n_clients):
        # Use your custom qudit CNOT gate
        control_qudit = qudits[i - 1]
        target_qudit = qudits[i]
        circuit.append(quditCNOTGate(d).on(control_qudit, target_qudit))
    return circuit, qudits


In [27]:
def teleportation_circuit(target_qudit, server_qudit, client_qudit, d):
    """Constructs the teleportation circuit for qudits."""
    circuit = cirq.Circuit()
    # Create the generalized Bell state between server and client
    circuit.append(quditHGate(d).on(server_qudit)) 
    circuit.append(quditCNOTGate(d).on(server_qudit, client_qudit)) 
    # Perform the Bell measurement between target qudit and server qudit
    circuit.append(quditCNOTGate(d).on(target_qudit, server_qudit))
    circuit.append(quditHGate(d).on(target_qudit))  
    circuit.append(cirq.measure(target_qudit, key=f"ell_{target_qudit.x}")) 
    circuit.append(cirq.measure(server_qudit, key=f"s_{server_qudit.x}"))  
    return circuit

In [28]:
def apply_teleportation_corrections(client_qudit, measurements, target_qudit_x, server_qudit_x, d):
    """Applies corrections based on teleportation measurements."""
    circuit = cirq.Circuit()
    s_key = f"s_{server_qudit_x}"
    ell_key = f"ell_{target_qudit_x}"
    s = int(measurements[s_key][0])  
    ell = int(measurements[ell_key][0]) 
    if s != 0:
        circuit.append((quditXGate(d) ** s).on(client_qudit))
    if ell != 0:
        circuit.append((quditZGate(d) ** ell).on(client_qudit))
    return circuit


In [29]:
def apply_local_operations(client_qudit, input_value, d):
    """Applies Z^{y_i} and Hadamard gate on the client's qudit."""
    circuit = cirq.Circuit()
    circuit.append((quditZGate(d) ** input_value).on(client_qudit))  # Z^{y_i}
    circuit.append(quditHGate(d).on(client_qudit))
    measurement_key = f"client_{client_qudit.x}"
    circuit.append(cirq.measure(client_qudit, key=measurement_key))
    return circuit

In [30]:
def server_debias_and_aggregate(measurements, gamma, kappa, n_clients, d):
    """Debiases and aggregates the measurements as per the protocol."""
    z = (-sum(measurements)) % d 
    unbiased_sum = (z - gamma * (kappa - 1) * n_clients / 2) / (1 - gamma)
    return unbiased_sum

In [31]:
# Prepare GHZ state
ghz_circuit, ghz_qudits = prepare_qudit_ghz_state(n_clients, d)
print(ghz_circuit)

0 (d=5): ───H(d=5)───C(d=5)─────────────────────
                     │
1 (d=5): ────────────X(d=5)───C(d=5)────────────
                              │
2 (d=5): ─────────────────────X(d=5)───C(d=5)───
                                       │
3 (d=5): ──────────────────────────────X(d=5)───


In [32]:
# Prepare teleportation circuits
client_qudits = []
teleport_circuits = []
for i in range(n_clients):
    server_qudit = cirq.LineQid(n_clients + i * 2, dimension=d)
    client_qudit = cirq.LineQid(n_clients + i * 2 + 1, dimension=d)
    client_qudits.append(client_qudit)
    
    # Generate teleportation circuit
    teleport_circuit = teleportation_circuit(
        target_qudit=ghz_qudits[i],
        server_qudit=server_qudit,
        client_qudit=client_qudit,
        d=d
    )
    teleport_circuits.append(teleport_circuit)

In [33]:
# Combine all circuits
total_circuit = cirq.Circuit()
total_circuit += ghz_circuit

for i in range(n_clients):
    total_circuit += teleport_circuits[i]
    
print(total_circuit)

0 (d=5): ────H(d=5)───C(d=5)───────────────────────────────────────C(d=5)───H(d=5)─────M('ell_0')─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                      │                                            │
1 (d=5): ─────────────X(d=5)───C(d=5)──────────────────────────────┼──────────────────────────────────────────────────C(d=5)───H(d=5)─────M('ell_1')──────────────────────────────────────────────────────────────────────────────────────────────────────────
                               │                                   │                                                  │
2 (d=5): ──────────────────────X(d=5)───C(d=5)─────────────────────┼──────────────────────────────────────────────────┼──────────────────────────────────────────────────C(d=5)───H(d=5)─────M('ell_2')───────────────────────────────────────────────────────
                                        │     

In [34]:
partial_result = simulator.run(total_circuit)

In [35]:
# Apply corrections based on measurements
for i in range(n_clients):
    server_qudit = cirq.LineQid(n_clients + i * 2, dimension=d)
    target_qudit = ghz_qudits[i]
    correction_circuit = apply_teleportation_corrections(
        client_qudit=client_qudits[i],
        measurements=partial_result.measurements,
        target_qudit_x=target_qudit.x,
        server_qudit_x=server_qudit.x,
        d=d
    )
    total_circuit += correction_circuit

C:\Users\svtsv\AppData\Local\Temp\ipykernel_40460\266554841.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  s = int(measurements[s_key][0])
C:\Users\svtsv\AppData\Local\Temp\ipykernel_40460\266554841.py:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ell = int(measurements[ell_key][0])


In [36]:
# Apply local operations for each client
for i, input_value in enumerate(inputs):
    client_qudit = client_qudits[i]
    local_circuit = apply_local_operations(client_qudit, input_value, d)
    total_circuit += local_circuit

In [37]:
# Simulate the entire protocol
final_result = simulator.run(total_circuit)

In [38]:
# Extract measurements
client_measurements = []
for i in range(n_clients):
    client_qudit = client_qudits[i]
    measurement_key = f"client_{client_qudit.x}"
    measurement = int(final_result.measurements[measurement_key][0])
    client_measurements.append(measurement)
    print(f"Measurement for Client {i+1}: {measurement}")

print("\nClient Measurements:", client_measurements)

Measurement for Client 1: 3
Measurement for Client 2: 2
Measurement for Client 3: 1
Measurement for Client 4: 2

Client Measurements: [3, 2, 1, 2]


C:\Users\svtsv\AppData\Local\Temp\ipykernel_40460\2544195267.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  measurement = int(final_result.measurements[measurement_key][0])


In [39]:
# Perform aggregation
aggregated_result = server_debias_and_aggregate(client_measurements, gamma, kappa, n_clients, d)
print("\nAggregated and Debiased Result at Server:", aggregated_result)


Aggregated and Debiased Result at Server: 2.0
